# HDF5

- Used for storing large data sets
- Supports storing a range of data types
- Heirarchical data format

More info can be found at http://www.hdfgroup.org/

In [1]:
# First time we need to do this
source("http://bioconductor.org/biocLite.R")
biocLite("rhdf5")

Installing package into 'C:/Users/emirl/Documents/R/win-library/3.3'
(as 'lib' is unspecified)


package 'BiocInstaller' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\emirl\AppData\Local\Temp\Rtmpao2aJp\downloaded_packages


Bioconductor version 3.3 (BiocInstaller 1.22.3), ?biocLite for help
BioC_mirror: https://bioconductor.org
Using Bioconductor 3.3 (BiocInstaller 1.22.3), R 3.3.1 (2016-06-21).
Installing package(s) 'rhdf5'
also installing the dependency 'zlibbioc'



package 'zlibbioc' successfully unpacked and MD5 sums checked
package 'rhdf5' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\emirl\AppData\Local\Temp\Rtmpao2aJp\downloaded_packages


Old packages: 'car', 'caret', 'cluster', 'codetools', 'colorspace', 'curl',
  'DBI', 'digest', 'dplyr', 'evaluate', 'foreign', 'htmlwidgets', 'httr',
  'IRdisplay', 'jsonlite', 'knitr', 'lattice', 'maps', 'Matrix', 'mgcv',
  'mime', 'pbdZMQ', 'quantmod', 'quantreg', 'R6', 'rbokeh', 'Rcpp',
  'RcppEigen', 'repr', 'rmarkdown', 'shiny', 'SparseM', 'stringi', 'stringr',
  'survival', 'tibble', 'tidyr'


- This will install packages from Bioconductor primarily used for genomics, but also has good "big data" packages.
- Can be used to interface with hdf5 datasets

In [2]:
library(rhdf5)
created <- h5createFile("example.h5")
created

[1] TRUE

### Create groups

In [4]:
created <- h5createGroup("example.h5", "foo")
created <- h5createGroup("example.h5", "bar")
created <- h5createGroup("example.h5", "foo/baz")
h5ls("example.h5")

,group,name,otype,dclass,dim
0,/,bar,H5I_GROUP,,
1,/,foo,H5I_GROUP,,
2,/foo,baz,H5I_GROUP,,


### Write to groups

In [6]:
A <- matrix(1:10, nrow = 5)
h5write(A, "example.h5", "foo/A")

In [7]:
# Multidim. array
B <- array(seq(0.1, 2.0, by=0.1), dim=c(5, 2, 2))
attr(B, "scale") <- "liter"
h5write(B, "example.h5", "foo/baz/N")

In [8]:
h5ls("example.h5")

,group,name,otype,dclass,dim
0,/,bar,H5I_GROUP,,
1,/,foo,H5I_GROUP,,
2,/foo,A,H5I_DATASET,INTEGER,5 x 2
3,/foo,baz,H5I_GROUP,,
4,/foo/baz,N,H5I_DATASET,FLOAT,5 x 2 x 2


### Write df to top level

In [18]:
df <- data.frame(1L:5L, seq(0, 1, length.out = 5), c("ab", "cde", "fghi", "a", "s"), stringsAsFactors=FALSE)
df

,X1L.5L,seq.0..1..length.out...5.,c..ab....cde....fghi....a....s..
1,1,0,ab
2,2,0.25,cde
3,3,0.5,fghi
4,4,0.75,a
5,5,1,s


In [21]:
h5write(df, "example.h5", "df")
h5ls("example.h5")

Warning message:
In h5checktypeOrOpenLoc(file, readonly = TRUE): An open HDF5 file handle exists. If the file has changed on disk meanwhile, the function may not work properly. Run 'H5close()' to close all open HDF5 object handles.

,group,name,otype,dclass,dim
0,/,bar,H5I_GROUP,,
1,/,df,H5I_DATASET,COMPOUND,5
2,/,foo,H5I_GROUP,,
3,/foo,A,H5I_DATASET,INTEGER,5 x 2
4,/foo,baz,H5I_GROUP,,
5,/foo/baz,N,H5I_DATASET,FLOAT,5 x 2 x 2


### Reading data

In [23]:
readA <- h5read("example.h5", "foo/A")
print(readA)

Warning message:
In h5checktypeOrOpenLoc(file, readonly = TRUE): An open HDF5 file handle exists. If the file has changed on disk meanwhile, the function may not work properly. Run 'H5close()' to close all open HDF5 object handles.

     [,1] [,2]
[1,]    1    6
[2,]    2    7
[3,]    3    8
[4,]    4    9
[5,]    5   10


### Write and reading chunks

In [24]:
h5write(c(12, 13, 14), "example.h5", "foo/A", index=list(1:3, 1)) # Insert new values to row 1-3 column 1
h5read("example.h5", "foo/A")

Warning message:
In h5checktypeOrOpenLoc(file): An open HDF5 file handle exists. If the file has changed on disk meanwhile, the function may not work properly. Run 'H5close()' to close all open HDF5 object handles.Warning message:
In h5checktypeOrOpenLoc(file, readonly = TRUE): An open HDF5 file handle exists. If the file has changed on disk meanwhile, the function may not work properly. Run 'H5close()' to close all open HDF5 object handles.

12,6
13,7
14,8
4,9
5,10
